# World CO2 Emissions Forecasting
## Fitting a LSTM Model (Recurrent Neural Network)
[*Cristian Castro Álvarez*](https://github.com/cristian-castro-a)

**Goal**: 
- To fit a LSTM Model to the World CO2 Emissions


**Data:**
- The data comes from [Our World in Data](https://github.com/owid/co2-data)
- Column 'CO2' of the dataframe indicates the Tonnes of CO2 emmitted into the atmosphere.
- The dataset includes yearly data from 1750 to 2020, with a total of 271 data points.

In [1]:
# Libraries
import math
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px
import plotly.graph_objects as go

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

mpl.rcParams['figure.figsize'] = (10,8)
mpl.rcParams['axes.grid'] = False

# The World CO2 Emissions Data

In [2]:
df = pd.read_csv('../Data/owid-co2-data.csv')
df.head()

,iso_code,country,year,co2,co2_per_capita,trade_co2,cement_co2,cement_co2_per_capita,coal_co2,coal_co2_per_capita,...,ghg_excluding_lucf_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,AFG,Afghanistan,1949,0.015,0.002,NaN,NaN,NaN,0.015,0.002,...,NaN,NaN,NaN,NaN,NaN,7624058.0,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1950,0.084,0.011,NaN,NaN,NaN,0.021,0.003,...,NaN,NaN,NaN,NaN,NaN,7752117.0,9.421400e+09,NaN,NaN,NaN
2,AFG,Afghanistan,1951,0.092,0.012,NaN,NaN,NaN,0.026,0.003,...,NaN,NaN,NaN,NaN,NaN,7840151.0,9.692280e+09,NaN,NaN,NaN
3,AFG,Afghanistan,1952,0.092,0.012,NaN,NaN,NaN,0.032,0.004,...,NaN,NaN,NaN,NaN,NaN,7935996.0,1.001732e+10,NaN,NaN,NaN
4,AFG,Afghanistan,1953,0.106,0.013,NaN,NaN,NaN,0.038,0.005,...,NaN,NaN,NaN,NaN,NaN,8039684.0,1.063052e+10,NaN,NaN,NaN


In [3]:
# Aggregate the data on a yearly basis (the entire world as one entity, I don't care about the emissions of individual countries)
df = df.groupby(by=['year']).sum().reset_index()[['year','co2']]
df.insert(loc = 1, column = 'month', value = 12)
df.insert(loc = 2, column = 'day', value = 31)
values = pd.to_datetime(df[['year','month','day']])
df.insert(loc = 0, column = 'date', value = values)
df.drop(['year','month','day'], axis = 1, inplace = True)
df.head()

,date,co2
0,1750-12-31,46.755
1,1751-12-31,46.755
2,1752-12-31,46.770
3,1753-12-31,46.770
4,1754-12-31,46.790


In [4]:
# To work with tonnes of CO2 it is necessary a conversion factor of 3.664
df['co2'] = df['co2']/3.664
df.tail().round(1)

,date,co2
266,2016-12-31,34035.4
267,2017-12-31,34471.8
268,2018-12-31,35058.0
269,2019-12-31,35049.9
270,2020-12-31,33185.8


In [37]:
# Visualizing the world emissions per year
fig = px.line(df, 
                x = 'date', 
                y = 'co2', 
                markers = True, 
                height = 800, 
                width = 1000)

fig.update_layout(title = dict(
        text = 'Total World CO2 Emissions',
        font = dict(
            family = 'Arial',
            size = 30
        ),
        x = 0.5
    )
    )

fig.update_traces(line_color = 'darkblue')

fig.update_xaxes(
    title_text = 'Date',
    title_font = {'size': 20}
)

fig.update_yaxes(
    title_text = 'Million Tonnes of CO2 Emmitted into the Atmosphere',
    title_font = {'size': 20}
)

fig.show()

# Preprocessing the Data for a LSTM Model

## Windowing Function

In [6]:
def df_to_inputs(df, window_size = 5):
    # Input: dataframe with the time series and window size
    # Windows size refers to the number of points in the series with which the model will be trained
    # Output: X, y for training
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        X.append([[a] for a in df_as_np[i:i+5]])
        y.append(df_as_np[i+5])
    return np.array(X), np.array(y)

In [7]:
# Getting the windows
WINDOW_SIZE = 5
X, y = df_to_inputs(df['co2'], WINDOW_SIZE)

In [8]:
X.shape, y.shape

((266, 5, 1), (266,))

In [9]:
X[1:3]

array([[[12.7606441 ],
        [12.76473799],
        [12.76473799],
        [12.77019651],
        [12.77565502]],

       [[12.76473799],
        [12.76473799],
        [12.77019651],
        [12.77565502],
        [13.65447598]]])

## Scaling and Splitting for Train, Validation and Test Sets

In [38]:
# 85% Train, 7.5% Valid, 7.5% Test
print(f'92.5% of samples for Training: {math.ceil(0.925*266)} samples')
print(f'7.5% of samples for Testing: {math.ceil(0.075*266)} samples')

92.5% of samples for Training: 247 samples
7.5% of samples for Testing: 20 samples


In [39]:
# 92.5% Train, 7.5% Test
df_train = df[:247].copy()
df_test = df[247:].copy()

In [12]:
# Scaling the Data (To have a range between 0 and 1 - Done only in the Train Set to avoid data leakage)
df_train_scaled = df_train
df_test_scaled = df_test

scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(df_train['co2'].values.reshape(-1,1))
df_train_scaled['co2'] = scaler.transform(df_train['co2'].values.reshape(-1,1))
df_test_scaled['co2'] = scaler.transform(df_test['co2'].values.reshape(-1,1))

In [13]:
# Windowing
WINDOW_SIZE = 5

X_train, y_train = df_to_inputs(df_train_scaled['co2'], WINDOW_SIZE)
X_test, y_test = df_to_inputs(df_test_scaled['co2'], WINDOW_SIZE)

In [14]:
# Checking right shapes
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((242, 5, 1), (242,), (19, 5, 1), (19,))

# Long Short Term Memory (LSTM) Model

## Model Construction

In [17]:
# Model Architecture as a results of long experimentation
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[1],1)))
model.add(tf.keras.layers.LSTM(units = 100, return_sequences = False))
model.add(tf.keras.layers.Dense(30))
model.add(tf.keras.layers.Dense(1))
model.summary()

2022-09-29 21:16:54.650911: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-29 21:16:54.651849: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5, 100)            40800     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 30)                3030      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 124,261
Trainable params: 124,261
Non-trainable params: 0
_________________________________________________________________


## Model Training

In [18]:
model.compile(optimizer = 'Adagrad', loss = 'mean_squared_error')
history = model.fit(X_train, y_train, batch_size = 1, epochs = 30)

Train on 242 samples
Epoch 1/30


2022-09-29 21:16:56.958594: W tensorflow/core/grappler/optimizers/implementation_selector.cc:310] Skipping optimization due to error while loading function libraries: Invalid argument: Functions '__inference___backward_standard_lstm_5014_5483' and '__inference___backward_standard_lstm_5014_5483_specialized_for_StatefulPartitionedCall_1_at___inference_distributed_function_5618' both implement 'lstm_a1795aa5-40f2-40d4-bef3-4c9000c15b5b' but their signatures do not match.


242/242 [==============================] - 8s 31ms/sample - loss: 0.0188
Epoch 2/30
242/242 [==============================] - 4s 17ms/sample - loss: 9.4045e-04
Epoch 3/30
242/242 [==============================] - 4s 16ms/sample - loss: 8.2927e-04
Epoch 4/30
242/242 [==============================] - 4s 17ms/sample - loss: 7.6812e-04
Epoch 5/30
242/242 [==============================] - 4s 16ms/sample - loss: 7.7092e-04
Epoch 6/30
242/242 [==============================] - 4s 16ms/sample - loss: 7.0293e-04
Epoch 7/30
242/242 [==============================] - 4s 16ms/sample - loss: 6.9544e-04
Epoch 8/30
242/242 [==============================] - 4s 16ms/sample - loss: 6.6388e-04
Epoch 9/30
242/242 [==============================] - 4s 16ms/sample - loss: 6.5583e-04
Epoch 10/30
242/242 [==============================] - 4s 16ms/sample - loss: 6.0510e-04
Epoch 11/30
242/242 [==============================] - 4s 16ms/sample - loss: 5.8085e-04
Epoch 12/30
242/242 [========================

In [19]:
# Visualizing the Loss
fig = px.line(df, 
                x = np.arange(1,len(history.history['loss'])+1), 
                y = history.history['loss'], 
                markers = True, 
                height = 500, 
                width = 700)

fig.update_layout(title = dict(
        text = 'Loss',
        font = dict(
            family = 'Arial',
            size = 20
        ),
        x = 0.5
    )
    )

fig.update_traces(line_color = 'darkblue')

fig.update_xaxes(
    title_text = 'Epoch',
    title_font = {'size': 15}
)

fig.update_yaxes(
    title_text = 'Loss',
    title_font = {'size': 15}
)

fig.show()

## Testing on the Test Set

In [20]:
# Predict and inverse the scaling
predicted = model.predict(X_test)
scaled_predicted = scaler.inverse_transform(predicted)
scaled_predicted

2022-09-29 21:18:57.489653: W tensorflow/core/grappler/optimizers/implementation_selector.cc:310] Skipping optimization due to error while loading function libraries: Invalid argument: Functions '__inference_standard_lstm_20842' and '__inference_standard_lstm_20842_specialized_for_sequential_lstm_1_StatefulPartitionedCall_at___inference_distributed_function_21150' both implement 'lstm_08c44725-b0f7-4455-9624-18c76e74cb68' but their signatures do not match.


array([[26471.482],
       [26723.895],
       [27214.371],
       [27870.143],
       [28572.467],
       [29470.88 ],
       [30494.88 ],
       [31370.248],
       [32023.438],
       [32608.273],
       [33120.074],
       [33590.1  ],
       [34080.727],
       [34927.367],
       [35372.625],
       [35574.33 ],
       [35681.81 ],
       [35816.51 ],
       [35982.41 ]], dtype=float32)

In [21]:
# Create a Dataframe for Predictions on the Test Set
df_pred = df[(len(df)-len(scaled_predicted)):]
df_pred['co2'] = scaled_predicted
df_pred.rename(columns = {'co2': 'co2_pred'}, inplace = True)

# Merge Dataframes on Date
result = pd.merge(df,df_pred, how = 'left', on = ['date'])
result.tail(20)

,date,co2,co2_pred
251,2001-12-31,25654.850164,NaN
252,2002-12-31,26152.109989,26471.482422
253,2003-12-31,27385.484170,26723.894531
254,2004-12-31,28474.503002,27214.371094
255,2005-12-31,29299.889465,27870.142578
256,2006-12-31,30190.065229,28572.466797
257,2007-12-31,30935.277020,29470.880859
258,2008-12-31,31392.197871,30494.880859
259,2009-12-31,30748.111627,31370.248047
260,2010-12-31,32367.336245,32023.437500


# Best Model: Visualizing the Results

In [50]:
# Plotting the Results
fig = go.Figure()

fig.update_layout(
    width = 1000,
    height = 800,
    title = dict(
        text = 'Total World CO2 Emissions: Real vs. Predicted',
        font = dict(
            family = 'Arial',
            size = 30
        ),
        x = 0.5
    )
)

fig.update_xaxes(
    title_text = 'Date',
    title_font = {'size': 20}
)

fig.update_yaxes(
    title_text = 'Million Tonnes of CO2 Emmitted into the Atmosphere',
    title_font = {'size': 20}
)

fig.add_trace(
    go.Scatter(
        x = result['date'],
        y = result['co2'],
        mode = 'lines',
        name = 'Ground Truth',
        marker = dict(
            color = 'darkblue',
            size = 15
        )
    )
)

fig.add_trace(
    go.Scatter(
        x = result['date'],
        y = result['co2_pred'],
        mode = 'lines',
        name = 'Predicted',
        marker = dict(
            color = 'orange',
            size = 15
        )
    )
)

fig.show()

## Performance of the Model in the Test Set: Mean Absolute Percentage Error (MAPE)

In [34]:
# Function for computing MAPE

def compute_mape(df, date_ini):
    # Input: dataframe with columns 'date', 'co2' and 'co2_pred', and date_ini.
    errors = []
    for ii in range(df.index[df.date == date_ini].tolist()[0],len(df)):
        date = df.iloc[ii]['date']
        y_true = df.iloc[ii]['co2']
        y_pred = df.iloc[ii]['co2_pred']
        mape = mean_absolute_error([y_true], [y_pred])/y_true
        errors.append([date, 100*mape])
    return errors

In [35]:
date_ini = result.loc[(len(df)-len(scaled_predicted)), 'date']
errors = compute_mape(result, date_ini)

In [47]:
# MAPE Graph
x, y = zip(*errors)

fig = go.Figure()

fig.update_layout(
    width = 1000,
    height = 800,
    title = dict(
        text = 'MAPE: Real vs. Predicted',
        font = dict(
            family = 'Arial',
            size = 30
        ),
        x = 0.5
    )
)

fig.update_xaxes(
    title_text = 'Date',
    title_font = {'size': 20}
)

fig.update_yaxes(
    title_text = 'Mean Absolute Error (%)',
    title_font = {'size': 20}
)

fig.add_trace(
    go.Scatter(
        x = x,
        y = y,
        mode = 'lines',
        marker = dict(
            color = 'darkblue',
            size = 15
        )
    )
)

fig.add_hline(y=5,
                line_width = 2,
                line_color = 'red',
                line_dash = 'dash'
)

fig.show()

**Comments:**
- The MAPE tends to stay below 5%
- Nonetheless, it shows a good trend into future predictions